In [1]:
import mysql.connector
import pymongo

In [19]:
conn = mysql.connector.connect(user='dev', password='dev', host='localhost', database='border')

In [20]:
def get_directions():
    dirdict = {}
    cur = conn.cursor()

    cur.execute("select id, name from direction")
    
    for id, name in cur:
        dirdict[name] = id
        
    cur.close()
    
    return dirdict

dirmap = get_directions()

In [21]:
def get_lanes():
    lanes = {}
    cur = conn.cursor()

    cur.execute("select id, name from lane")
    
    for id, name in cur:
        lanes[name] = id
        
    cur.close()
    
    return lanes

lanemap = get_lanes()

In [22]:
client = pymongo.MongoClient()
db = client.border

In [31]:
for crossing in db.crossing.find():
    for lane in crossing['LaneTypes']:
        for direction in crossing['DirectionsOfTravel']:
            insert_crossing(crossing['ID'], lanemap[lane], dirmap[direction])


In [13]:
def insert_crossing(location_id, lane_id, direction_id):
    cur = conn.cursor()

    cur.execute("INSERT INTO crossing (location_id, lane_id, direction_id) VALUES (%s, %s, %s)",\
            (location_id, lane_id, direction_id))
    
    conn.commit()
    cur.close()

In [32]:
conn.close()